# Setup

In [1]:
import sys
import importlib

sys.path.insert(0, "../PopV")
import popv
from popv.reproducibility import _accuracy
importlib.reload(_accuracy)

2023-03-12 11:48:17.041248: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 11:48:17.212165: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-12 11:48:17.831579: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-03-12 11:48:17.831635: W tensorflow/compiler/xla/stream_executor/platf

<module 'popv.reproducibility._accuracy' from '/data/extra_files/galen_dropbox_figures/can_reproducibility/thymus_models/../PopV/popv/reproducibility/_accuracy.py'>

In [2]:
import anndata
import scanpy as sc
import numpy as np
import obonet
import networkx as nx
import pandas as pd
import time

In [3]:
from popv._utils import make_ontology_dag

In [4]:
obo_fn = '../PopV/ontology/cl.obo'

G = popv._utils.make_ontology_dag(obo_fn)

# Thymus


In [5]:
fn = 'thymus_annotation/annotation/query_and_reference_popv.h5ad'
adata = anndata.read(fn)

In [6]:
adata

AnnData object with n_obs × n_vars = 289562 × 4000
    obs: 'tissue_in_publication', 'donor_id', 'free_annotation', 'compartment', 'cell_type_ontology_term_id', 'cell_type', 'assay', 'sex', 'tissue', 'cell_ontology_class', 'cell_ontology_class_tissue', 'donor_tissue', 'assay_correct', 'donor_assay', 'cell_ontology_class_original', '_batch_annotation', '_labels_annotation', '_ref_subsample', 'Sample', 'paper_annotation', 'development_stage', '_dataset', 'n_counts', 'popv_celltypist_prediction', 'popv_celltypist_prediction_probabilities', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_bbknn_prediction_probabilities', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scanorama_prediction_probabilities', 'subsampled_labels', '_scvi_batch', '_scvi_labels', 'popv_knn_on_scvi_prediction', 'popv_knn_on_scvi_prediction_probabilities', '_labels_annotation_cell_ontology_id', 'popv_onclass_prediction', 'popv_onclass_seen', 'popv_onclass_prediction_probabilities', 'popv_onclass_seen_probabilities', 'p

# Add seurat results

In [7]:
df = pd.read_csv('thymus_annotation/seurat_annotation/seurat_predictions.csv')

In [8]:
df


,Unnamed: 0,predicted.id,prediction.score.CD4.positive.helper.T.cell,prediction.score.naive.regulatory.T.cell,prediction.score.T.follicular.helper.cell,prediction.score.CD8.positive..alpha.beta.cytotoxic.T.cell,prediction.score.DN4.thymocyte,prediction.score.B.cell,prediction.score.medullary.thymic.epithelial.cell,prediction.score.fibroblast,...,prediction.score.CD8.positive..alpha.beta.T.cell,prediction.score.memory.B.cell,prediction.score.naive.B.cell,prediction.score.mesothelial.cell,prediction.score.fast.muscle.cell,prediction.score.dendritic.cell,prediction.score.immature.natural.killer.cell,prediction.score.thymocyte,prediction.score.mast.cell,prediction.score.max
0,FCAImmP7179369-AAACCTGAGCCCAATT,CD4-positive helper T cell,0.652273,0.031506,0.000000,0.055114,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.030866,0.0,0.652273
1,FCAImmP7179369-AAACCTGAGCCTATGT,CD4-positive helper T cell,0.485668,0.276913,0.106205,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.485668
2,FCAImmP7179369-AAACCTGAGTCGCCGT,naive regulatory T cell,0.221118,0.306386,0.000000,0.000000,0.093813,0.0,0.0,0.0,...,0.000000,0.008232,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.306386
3,FCAImmP7179369-AAACCTGCAGCATGAG,CD4-positive helper T cell,0.620940,0.054479,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.065298,0.000000,0.0,0.0,0.0,0.000000,0.0,0.013377,0.0,0.620940
4,FCAImmP7179369-AAACCTGGTCTCCCTA,CD4-positive helper T cell,0.685225,0.000000,0.009169,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.148957,0.0,0.685225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255896,Human_colon_16S7985397-TTTGGTTCAAACCTAC,DN4 thymocyte,0.044308,0.000000,0.000000,0.000000,0.852812,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.852812
255897,Human_colon_16S7985397-TTTGGTTCATGGTAGG,CD4-positive helper T cell,0.515515,0.010717,0.003768,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.436151,0.0,0.515515
255898,Human_colon_16S7985397-TTTGGTTTCAAGGTAA,DN4 thymocyte,0.145259,0.085304,0.000000,0.000000,0.358652,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.017198,0.0,0.000000,0.0,0.358652
255899,Human_colon_16S7985397-TTTGTCAAGCTGAACG,CD4-positive helper T cell,0.621363,0.146297,0.017105,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.068841,0.0,0.621363


In [9]:
df['seurat_prediction'] = df["predicted.id"]
df['seurat_probability'] = df["prediction.score.max"]

In [10]:
df.index = df["Unnamed: 0"]

In [11]:
adata.obs['seurat_prediction'] = adata.obs['cell_ontology_class'].astype(str)
adata.obs['seurat_probability'] = 1

In [12]:
adata.obs.loc[adata.obs['_dataset']=='query', 'seurat_prediction'] = df.loc[adata[adata.obs['_dataset']=='query'].obs_names, 'seurat_prediction']
adata.obs.loc[adata.obs['_dataset']=='query', 'seurat_probability'] = df.loc[adata[adata.obs['_dataset']=='query'].obs_names, 'seurat_probability']

In [13]:
adata.obs['seurat_prediction']

4861STDY7462263-GGTGAAGCACGGCTAC                           CD4-positive helper T cell
4861STDY7462261-CCTAGCTGTCCAACTA                           CD4-positive helper T cell
4861STDY7462261-CCTAGCTGTAGCTGCC                           CD4-positive helper T cell
4861STDY7462261-CCTAGCTCATGTAGTC                           CD4-positive helper T cell
4861STDY7462261-CCTAGCTCAGCTGTGC                                               B cell
                                                                      ...            
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_O7_S283               endothelial cell
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_O8_S284               endothelial cell
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_P10_S10               endothelial cell
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_P13_S13               endothelial cell
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_J17_S173              endothelial cell
Name: seurat_prediction, Length: 289562, dtype: object

In [14]:
adata.obs.replace({'CD8-alpha-alpha-positive, intraepithelial T cell': 'CD8-alpha-alpha-positive, alpha-beta intraepithelial T cell'}, inplace=True)

## Fine Accuracy

### ontology_vote

In [15]:
pred_keys = list(set(adata.uns['prediction_keys']).union(set(adata.uns['prediction_keys_seen']))) + ['seurat_prediction']

cts = np.unique(adata.obs[pred_keys])

bad_cts = []

for ct in cts:
    if ct not in G:
        bad_cts.append(ct)
    elif 'cell' not in nx.shortest_path_length(G, ct):
        bad_cts.append(ct)
        
cts = np.unique(adata.obs['cell_ontology_class'])

for ct in cts:
    if ct not in G:
        print('2', ct)
        bad_cts.append(ct)
    elif 'cell' not in nx.shortest_path_length(G, ct):
        print('1', ct)
        bad_cts.append(ct)

In [16]:
pred_keys

['popv_onclass_prediction',
 'popv_scanvi_prediction',
 'popv_celltypist_prediction',
 'popv_svm_prediction',
 'popv_knn_on_bbknn_prediction',
 'popv_rf_prediction',
 'popv_knn_on_scvi_prediction',
 'popv_knn_on_scanorama_prediction',
 'popv_onclass_seen',
 'seurat_prediction']

In [17]:
# should be empty

bad_cts

[]

In [18]:
_accuracy._fine_ontology_sibling_accuracy(
    adata, 
    obofile=obo_fn,
    gt_key='cell_ontology_class', 
    pred_key='popv_prediction')

In [19]:
adata.obs

,tissue_in_publication,donor_id,free_annotation,compartment,cell_type_ontology_term_id,cell_type,assay,sex,tissue,cell_ontology_class,...,popv_majority_vote_prediction,popv_majority_vote_score,popv_prediction,popv_prediction_score,popv_prediction_depth,popv_prediction_onclass_relative_depth,popv_parent,seurat_prediction,seurat_probability,popv_prediction_ontology_distance
4861STDY7462263-GGTGAAGCACGGCTAC,NaN,A16,NaN,NaN,NaN,NaN,10x 3' v2,female,thymus,naive regulatory T cell,...,CD4-positive helper T cell,6,CD4-positive helper T cell,5,11,-1,"CD4-positive, alpha-beta T cell",CD4-positive helper T cell,0.836335,3_sib
4861STDY7462261-CCTAGCTGTCCAACTA,NaN,A16,NaN,NaN,NaN,NaN,10x 3' v2,female,thymus,"double-positive, alpha-beta thymocyte",...,thymocyte,5,thymocyte,5,9,1,thymocyte,CD4-positive helper T cell,0.561692,0
4861STDY7462261-CCTAGCTGTAGCTGCC,NaN,A16,NaN,NaN,NaN,NaN,10x 3' v2,female,thymus,naive regulatory T cell,...,CD4-positive helper T cell,6,CD4-positive helper T cell,5,11,-5,T cell,CD4-positive helper T cell,0.697373,3_sib
4861STDY7462261-CCTAGCTCATGTAGTC,NaN,A16,NaN,NaN,NaN,NaN,10x 3' v2,female,thymus,"double-positive, alpha-beta thymocyte",...,CD4-positive helper T cell,4,CD4-positive helper T cell,3,11,-5,T cell,CD4-positive helper T cell,0.478020,5_sib
4861STDY7462261-CCTAGCTCAGCTGTGC,NaN,A16,NaN,NaN,NaN,NaN,10x 3' v2,female,thymus,B cell,...,B cell,8,B cell,7,8,-1,B cell,B cell,0.855541,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_O7_S283,Thymus,TSP2,capillary endothelial cell,endothelial,CL:0002144,capillary endothelial cell,Smart-seq2,female,thymus,endothelial cell,...,endothelial cell,8,endothelial cell,8,5,1,endothelial cell,endothelial cell,1.000000,0
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_O8_S284,Thymus,TSP2,vein endothelial cell,endothelial,CL:0002543,vein endothelial cell,Smart-seq2,female,thymus,endothelial cell,...,endothelial cell,8,endothelial cell,8,5,1,endothelial cell,endothelial cell,1.000000,0
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_P10_S10,Thymus,TSP2,capillary endothelial cell,endothelial,CL:0002144,capillary endothelial cell,Smart-seq2,female,thymus,endothelial cell,...,endothelial cell,8,endothelial cell,8,5,1,endothelial cell,endothelial cell,1.000000,0
TSP2_Thymus_NA_SS2_B113470_B134459_Endothelial_P13_S13,Thymus,TSP2,capillary endothelial cell,endothelial,CL:0002144,capillary endothelial cell,Smart-seq2,female,thymus,endothelial cell,...,endothelial cell,8,endothelial cell,8,5,1,endothelial cell,endothelial cell,1.000000,0


### all other methods

In [20]:
pred_keys

['popv_onclass_prediction',
 'popv_scanvi_prediction',
 'popv_celltypist_prediction',
 'popv_svm_prediction',
 'popv_knn_on_bbknn_prediction',
 'popv_rf_prediction',
 'popv_knn_on_scvi_prediction',
 'popv_knn_on_scanorama_prediction',
 'popv_onclass_seen',
 'seurat_prediction']

In [21]:
for k in pred_keys:
    print(k)
    _accuracy._fine_ontology_sibling_accuracy(
        adata, 
        obofile=obo_fn,
        gt_key='cell_ontology_class', 
        pred_key=k)

popv_onclass_prediction
popv_scanvi_prediction
popv_celltypist_prediction
popv_svm_prediction
popv_knn_on_bbknn_prediction
popv_rf_prediction
popv_knn_on_scvi_prediction
popv_knn_on_scanorama_prediction
popv_onclass_seen
seurat_prediction


In [22]:
adata.write('thymus_annotation/annotation/query_and_reference_popv_fine_ontology_accuracy.h5ad')

... storing 'seurat_prediction' as categorical


## Coarse Accuracy

### ontology_vote

In [23]:
adata.obs.drop(labels=[i for i in adata.obs.columns if 'ontology_distance' in i], axis=1, inplace=True)
adata

AnnData object with n_obs × n_vars = 289562 × 4000
    obs: 'tissue_in_publication', 'donor_id', 'free_annotation', 'compartment', 'cell_type_ontology_term_id', 'cell_type', 'assay', 'sex', 'tissue', 'cell_ontology_class', 'cell_ontology_class_tissue', 'donor_tissue', 'assay_correct', 'donor_assay', 'cell_ontology_class_original', '_batch_annotation', '_labels_annotation', '_ref_subsample', 'Sample', 'paper_annotation', 'development_stage', '_dataset', 'n_counts', 'popv_celltypist_prediction', 'popv_celltypist_prediction_probabilities', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_bbknn_prediction_probabilities', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scanorama_prediction_probabilities', 'subsampled_labels', '_scvi_batch', '_scvi_labels', 'popv_knn_on_scvi_prediction', 'popv_knn_on_scvi_prediction_probabilities', '_labels_annotation_cell_ontology_id', 'popv_onclass_prediction', 'popv_onclass_seen', 'popv_onclass_prediction_probabilities', 'popv_onclass_seen_probabilities', 'p

In [24]:
_accuracy._ontology_accuracy(
    adata, 
    obofile=obo_fn,
    gt_key='cell_ontology_class', 
    pred_key='popv_prediction',
    save_key=f'consensus_ontology_accuracy')

In [25]:
adata.obs['consensus_ontology_accuracy'].value_counts()

no match    120399
exact        81772
parent       66474
child        11280
sibling       9637
Name: consensus_ontology_accuracy, dtype: int64

In [26]:
pd.set_option('display.max_rows', 500)

In [27]:
df = adata.obs[adata.obs['consensus_ontology_accuracy']=='no match'].copy()
df['Count'] = 1
df = df.groupby(['popv_prediction', 'cell_ontology_class']).Count.count().reset_index()
df[df['Count']>10]

,popv_prediction,cell_ontology_class,Count
11,B cell,cortical thymic epithelial cell,12
12,B cell,dendritic cell,316
13,B cell,double negative thymocyte,225
22,B cell,innate lymphoid cell,66
27,B cell,medullary thymic epithelial cell,33
33,B cell,plasmacytoid dendritic cell,952
34,B cell,thymocyte,132
38,CD4-positive helper T cell,"CD8-alpha-alpha-positive, alpha-beta intraepit...",1624
39,CD4-positive helper T cell,"CD8-positive, alpha-beta T cell",7544
41,CD4-positive helper T cell,DN1 thymic pro-T cell,91


### all other methods

In [28]:
pred_keys

['popv_onclass_prediction',
 'popv_scanvi_prediction',
 'popv_celltypist_prediction',
 'popv_svm_prediction',
 'popv_knn_on_bbknn_prediction',
 'popv_rf_prediction',
 'popv_knn_on_scvi_prediction',
 'popv_knn_on_scanorama_prediction',
 'popv_onclass_seen',
 'seurat_prediction']

In [29]:
for k in pred_keys:
    print(k)
    _accuracy._ontology_accuracy(
        adata, 
        obofile=obo_fn,
        gt_key='cell_ontology_class', 
        pred_key=k,
        save_key=f'{k}_ontology_accuracy')

popv_onclass_prediction
popv_scanvi_prediction
popv_celltypist_prediction
popv_svm_prediction
popv_knn_on_bbknn_prediction
popv_rf_prediction
popv_knn_on_scvi_prediction
popv_knn_on_scanorama_prediction
popv_onclass_seen
seurat_prediction


In [30]:
adata.write('thymus_annotation/annotation/query_and_reference_final.h5ad')

... storing 'consensus_ontology_accuracy' as categorical
... storing 'popv_onclass_prediction_ontology_accuracy' as categorical
... storing 'popv_scanvi_prediction_ontology_accuracy' as categorical
... storing 'popv_celltypist_prediction_ontology_accuracy' as categorical
... storing 'popv_svm_prediction_ontology_accuracy' as categorical
... storing 'popv_knn_on_bbknn_prediction_ontology_accuracy' as categorical
... storing 'popv_rf_prediction_ontology_accuracy' as categorical
... storing 'popv_knn_on_scvi_prediction_ontology_accuracy' as categorical
... storing 'popv_knn_on_scanorama_prediction_ontology_accuracy' as categorical
... storing 'popv_onclass_seen_ontology_accuracy' as categorical
... storing 'seurat_prediction_ontology_accuracy' as categorical
